In [104]:
import pandas as pd
import numpy as np
#import openpyxl
import os
from tqdm import tqdm
tqdm.pandas()
import csv

import tensorflow as tf
#from tensorflow.keras.models import Sequential
#from tensorflow.keras.layers import Dense
#from tensorflow.keras.utils import to_categorical

from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
import tensorflow_decision_forests as tfdf
import random
import matplotlib.pyplot as plt

In [105]:
INPUT_FILE_PATH = "../data/5_lichess_csv_ML_ready/Elo_1500_1600.csv.pkl"


In [106]:
df = pd.read_pickle(INPUT_FILE_PATH)

#! remove draws for now
df = df[df['Result'] != 2]

#! remove unwanted columns in the training
df = df.drop(columns=['Board','MoveNumber','WhiteElo','BlackElo'])
#df = df.reset_index(drop=True)
#df = df.reindex(sorted(df.columns), axis=1)

In [107]:
df

,Result,K_white,Q_white,R_white,B_white,N_white,P_white,k_black,q_black,r_black,...,King_zone_controlled_squares_white,King_zone_controlled_squares_black,Trapped_bishops_white,Trapped_bishops_black,Trapped_rooks_white,Trapped_rooks_black,Trapped_queens_white,Trapped_queens_black,Check_white,Check_black
Id,,,,,,,,,,,,,,,,,,,,,
https://lichess.org/PxjUxB45-6,0.0,1,1,2,2,2,8,1,1,2,...,5,5,1,1,2,2,1,1,0,0
https://lichess.org/PxjUxB45-7,0.0,1,1,2,2,2,8,1,1,2,...,5,5,1,1,2,2,1,1,0,0
https://lichess.org/PxjUxB45-8,0.0,1,1,2,2,2,8,1,1,2,...,5,5,1,1,2,2,1,0,0,0
https://lichess.org/PxjUxB45-9,0.0,1,1,2,2,2,8,1,1,2,...,5,5,1,1,2,2,0,0,0,0
https://lichess.org/PxjUxB45-10,0.0,1,1,2,2,2,7,1,1,2,...,5,5,1,1,2,2,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
https://lichess.org/c6mXMJ23-44,0.0,1,0,2,0,2,6,1,0,2,...,4,5,0,0,1,2,0,0,0,0
https://lichess.org/c6mXMJ23-45,0.0,1,0,2,0,2,6,1,0,1,...,4,4,0,0,1,0,0,0,0,0
https://lichess.org/c6mXMJ23-46,0.0,1,0,2,0,1,6,1,0,1,...,4,4,0,0,1,1,0,0,0,0


In [108]:
df.index

Index(['https://lichess.org/PxjUxB45-6', 'https://lichess.org/PxjUxB45-7',
       'https://lichess.org/PxjUxB45-8', 'https://lichess.org/PxjUxB45-9',
       'https://lichess.org/PxjUxB45-10', 'https://lichess.org/PxjUxB45-11',
       'https://lichess.org/PxjUxB45-12', 'https://lichess.org/PxjUxB45-13',
       'https://lichess.org/PxjUxB45-14', 'https://lichess.org/PxjUxB45-15',
       ...
       'https://lichess.org/c6mXMJ23-39', 'https://lichess.org/c6mXMJ23-40',
       'https://lichess.org/c6mXMJ23-41', 'https://lichess.org/c6mXMJ23-42',
       'https://lichess.org/c6mXMJ23-43', 'https://lichess.org/c6mXMJ23-44',
       'https://lichess.org/c6mXMJ23-45', 'https://lichess.org/c6mXMJ23-46',
       'https://lichess.org/c6mXMJ23-47', 'https://lichess.org/c6mXMJ23-48'],
      dtype='object', name='Id', length=3755075)

In [109]:
#check all unique values in df.index, ignore the characters after an hypen
df.index = df.index.str.split('-').str[0]

In [110]:
games = df.index.unique()

In [111]:
games = games.tolist()
train_ids = random.sample(games, k=round(len(games) * 0.7))
games = list(set(games) - set(train_ids))
validation_ids = random.sample(games, k=round(len(games) * 0.5))
test_ids = list(set(games) - set(validation_ids))

In [112]:
len(train_ids)

46115

In [115]:
df.loc[train_ids]

,Result,K_white,Q_white,R_white,B_white,N_white,P_white,k_black,q_black,r_black,...,King_zone_controlled_squares_white,King_zone_controlled_squares_black,Trapped_bishops_white,Trapped_bishops_black,Trapped_rooks_white,Trapped_rooks_black,Trapped_queens_white,Trapped_queens_black,Check_white,Check_black
Id,,,,,,,,,,,,,,,,,,,,,
https://lichess.org/tFHrtnDb,0.0,1,1,2,2,2,7,1,1,2,...,5,5,2,2,2,2,0,0,0,0
https://lichess.org/tFHrtnDb,0.0,1,1,2,2,2,7,1,1,2,...,5,5,2,2,2,2,0,0,0,0
https://lichess.org/tFHrtnDb,0.0,1,1,2,2,2,7,1,1,2,...,5,5,2,1,2,2,0,0,0,0
https://lichess.org/tFHrtnDb,0.0,1,1,2,2,2,7,1,1,2,...,5,5,2,1,1,2,0,0,0,0
https://lichess.org/tFHrtnDb,0.0,1,1,2,2,2,7,1,1,2,...,5,5,2,1,1,2,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
https://lichess.org/kUz6HDn1,1.0,1,1,0,0,0,3,1,1,1,...,4,4,0,0,0,0,0,0,0,0
https://lichess.org/kUz6HDn1,1.0,1,1,0,0,0,3,1,1,1,...,4,4,0,0,0,0,0,0,1,0
https://lichess.org/kUz6HDn1,1.0,1,1,0,0,0,3,1,1,1,...,5,4,0,0,0,0,0,0,0,0
